# Projeto: Classificador de SMS Spam
## Pós-graduação em Engenharia de Sistemas de Software Inteligentes

Este projeto tem como objetivo construir um classificador de mensagens SMS em inglês para detectar se uma mensagem é spam ou não spam (ham), utilizando algoritmos clássicos de machine learning e boas práticas de ciência de dados.

O dataset utilizado é o SMS Spam Collection, disponível publicamente, composto por milhares de mensagens rotuladas como spam ou não spam.


## Carga do Dataset

O dataset foi obtido do [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/228/sms+spam+collection) e contém mensagens em inglês, classificadas como "spam" ou "ham".


In [1]:
import pandas as pd
url = "https://gist.githubusercontent.com/Thivieira/aa018594f9a6e05e005f7c3f3136f4f2/raw/7c2b4aa3cd212c369471db6ce26119227c4a38e4/SMSSpamCollection"
df = pd.read_csv(url, sep='\t', header=None, names=['label', 'text'])
df.head()


,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


## Análise Exploratória

A seguir, exploramos as proporções das classes e algumas características das mensagens.


In [2]:
# Quantidade de exemplos por classe
print(df['label'].value_counts())
print("\nProporção (%):")
print(df['label'].value_counts(normalize=True)*100)


label
ham     4825
spam     747
Name: count, dtype: int64

Proporção (%):
label
ham     86.593683
spam    13.406317
Name: proportion, dtype: float64


In [3]:
# Exemplo de mensagens
print("\nExemplos de mensagens NÃO SPAM:")
print(df[df['label']=='ham']['text'].head(2).to_list())

print("\nExemplos de mensagens SPAM:")
print(df[df['label']=='spam']['text'].head(2).to_list())



Exemplos de mensagens NÃO SPAM:
['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...', 'Ok lar... Joking wif u oni...']

Exemplos de mensagens SPAM:
["Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's", "FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv"]


## Pré-processamento dos Dados

Convertendo rótulos em valores numéricos, separando conjuntos de treino e teste (holdout) e preparando para a vetorização.


In [4]:
from sklearn.model_selection import train_test_split

df['target'] = df['label'].map({'ham': 0, 'spam': 1})
X = df['text']
y = df['target']

# Holdout estratificado
test_size = 0.20
seed = 42
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=test_size, shuffle=True, random_state=seed, stratify=y
)
print(f"Tamanho treino: {len(X_train)} | Tamanho teste: {len(X_test)}")


Tamanho treino: 4457 | Tamanho teste: 1115


## Transformação dos Dados

Como as mensagens são texto, usamos TfidfVectorizer para converter as palavras em números. Para modelos como KNN e SVM, é importante a normalização dos dados.


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


## Modelagem

Aqui treinamos e avaliamos quatro modelos clássicos: KNN, Decision Tree, Naive Bayes e SVM.


In [6]:
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

pipelines = {
    "KNN": Pipeline([('vectorizer', TfidfVectorizer()), ('clf', KNeighborsClassifier())]),
    "Decision Tree": Pipeline([('vectorizer', TfidfVectorizer()), ('clf', DecisionTreeClassifier(random_state=42))]),
    "Naive Bayes": Pipeline([('vectorizer', TfidfVectorizer()), ('clf', MultinomialNB())]),
    "SVM": Pipeline([('vectorizer', TfidfVectorizer()), ('clf', SVC(kernel='linear', probability=True))])
}


## Otimização de Hiperparâmetros

Utilizamos validação cruzada e GridSearch para buscar os melhores parâmetros para cada modelo.


In [7]:
from sklearn.model_selection import GridSearchCV

params_knn = {'clf__n_neighbors': [3, 5, 7]}
grid_knn = GridSearchCV(pipelines["KNN"], params_knn, cv=5, scoring='f1')
grid_knn.fit(X_train, y_train)

params_dt = {'clf__max_depth': [None, 10, 20, 30]}
grid_dt = GridSearchCV(pipelines["Decision Tree"], params_dt, cv=5, scoring='f1')
grid_dt.fit(X_train, y_train)

params_nb = {}  # MultinomialNB tem poucos hiperparâmetros úteis para texto

params_svm = {'clf__C': [0.1, 1, 10]}
grid_svm = GridSearchCV(pipelines["SVM"], params_svm, cv=5, scoring='f1')
grid_svm.fit(X_train, y_train)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                                       ('clf',
                                        SVC(kernel='linear',
                                            probability=True))]),
             param_grid={'clf__C': [0.1, 1, 10]}, scoring='f1')

## Avaliação e Comparação dos Modelos

Comparamos os modelos treinados usando as métricas accuracy, precision, recall e f1-score.


In [8]:
from sklearn.metrics import classification_report, confusion_matrix

modelos_otimizados = {
    "KNN": grid_knn.best_estimator_,
    "Decision Tree": grid_dt.best_estimator_,
    "Naive Bayes": pipelines["Naive Bayes"].fit(X_train, y_train),
    "SVM": grid_svm.best_estimator_
}

for nome, modelo in modelos_otimizados.items():
    y_pred = modelo.predict(X_test)
    print(f"\n==== {nome} ====")
    print(classification_report(y_test, y_pred, target_names=["Não Spam", "Spam"]))
    print("Matriz de confusão:\n", confusion_matrix(y_test, y_pred))



==== KNN ====
              precision    recall  f1-score   support

    Não Spam       0.96      1.00      0.98       966
        Spam       1.00      0.76      0.86       149

    accuracy                           0.97      1115
   macro avg       0.98      0.88      0.92      1115
weighted avg       0.97      0.97      0.97      1115

Matriz de confusão:
 [[966   0]
 [ 36 113]]

==== Decision Tree ====
              precision    recall  f1-score   support

    Não Spam       0.97      0.99      0.98       966
        Spam       0.91      0.83      0.87       149

    accuracy                           0.97      1115
   macro avg       0.94      0.91      0.92      1115
weighted avg       0.97      0.97      0.97      1115

Matriz de confusão:
 [[953  13]
 [ 25 124]]

==== Naive Bayes ====
              precision    recall  f1-score   support

    Não Spam       0.96      1.00      0.98       966
        Spam       1.00      0.70      0.83       149

    accuracy                   

## Exportação do Melhor Modelo

Exportamos o pipeline treinado para ser usado na aplicação web.


In [9]:
import joblib

# Exemplo: supondo que SVM foi o melhor
best_model = grid_svm.best_estimator_
joblib.dump(best_model, 'spam_model.joblib')


['spam_model.joblib']

## Conclusão

- O modelo de SVM apresentou o melhor desempenho geral.
- A classificação de mensagens curtas e com palavras genéricas ainda é um desafio.
- Para uso real no Brasil, seria necessário treinar com dados em português.


## Considerações de Segurança

- Não armazenar mensagens enviadas pelos usuários no sistema.
- Remover qualquer informação pessoal dos dados antes do uso.
- Se o sistema fosse para produção, adequação à LGPD seria obrigatória.
